In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.simplefilter("ignore")


def combining_bets(where):
    path = where
    files = os.listdir(path)

    bets_combined = pd.concat([pd.read_excel(
        f'{path}/{file}') for file in files if file.endswith(('.xlsx', '.xls'))], ignore_index=True)

    return bets_combined


path_bets = '../../db/bets/'
all_bets = combining_bets(path_bets)


all_bets.shape
print(all_bets.describe(include='all'))

                WTA   Location Tournament                           Date  \
count   4845.000000       4845       4845                           4845   
unique          NaN         67         72                            NaN   
top             NaN  Melbourne  Wimbledon                            NaN   
freq            NaN        316        254                            NaN   
mean      26.837564        NaN        NaN  2022-12-06 18:01:33.622290944   
min        1.000000        NaN        NaN            2022-01-03 00:00:00   
25%       14.000000        NaN        NaN            2022-06-14 00:00:00   
50%       25.000000        NaN        NaN            2023-01-06 00:00:00   
75%       40.000000        NaN        NaN            2023-06-15 00:00:00   
max       56.000000        NaN        NaN            2023-10-29 00:00:00   
std       15.091225        NaN        NaN                            NaN   

          Tier    Court Surface      Round  Best of      Winner  ...  \
count     4845 

In [2]:
all_bets_completed = all_bets[all_bets['Comment'] == 'Completed']


columns_to_keep = ['Date', 'Winner', 'Loser', 'AvgW', 'AvgL']


minimal_bets = all_bets_completed[columns_to_keep]

minimal_bets.rename(columns={'Date': 'date'}, inplace=True)

minimal_bets.dropna(subset=['AvgW', 'AvgL'], inplace=True)

minimal_bets.head()

,date,Winner,Loser,AvgW,AvgL
0,2022-01-03,Sakkari M.,Zidansek T.,1.13,5.75
1,2022-01-03,Rybakina E.,Sanders S.,1.20,4.40
2,2022-01-03,Juvan K.,Paquet C.,1.45,2.71
3,2022-01-03,Tomljanovic A.,Watson H.,1.30,3.45
4,2022-01-03,Fernandez L.A.,Alexandrova E.,1.65,2.21


In [3]:
minimal_bets.describe(include='all')

minimal_bets['match_id'] = minimal_bets.index + 1

In [4]:
import random

def bets_result_scrambler(database):

    db_with_wins = pd.DataFrame()

    for id, match in database.iterrows():
        date = match['date']
        sudo_random_number = random.randint(0, 100)

        p1 = pd.DataFrame({
            'match_id': match['match_id'],
            'name': match['Winner'],
            'bets': match['AvgW']}, index=[0])

        p2 = pd.DataFrame({
            'match_id': match['match_id'],
            'name': match['Loser'],
            'bets': match['AvgL']}, index=[0])

        if sudo_random_number % 2 == 0:
            # player 1 == winner y = 1
            match_vitals = pd.DataFrame({
                'match_id': match['match_id'],
                'date': date,
                'y': 1
            }, index=[0])
            player_stats_merged = pd.merge(
                p1, p2, on='match_id', suffixes=('_P1', '_P2'))
            player_stats_merged = pd.merge(
                match_vitals, player_stats_merged, on='match_id')
            db_with_wins = pd.concat(
                [player_stats_merged, db_with_wins], ignore_index=True)

        else:
            # player 2 == winner y = 0
            match_vitals = pd.DataFrame({
                'match_id': match['match_id'],
                'date': date,
                'y': 0
            }, index=[0])
            player_stats_merged = pd.merge(
                p2, p1, on='match_id', suffixes=('_P1', '_P2'))
            player_stats_merged = pd.merge(
                match_vitals, player_stats_merged, on='match_id')
            db_with_wins = pd.concat(
                [player_stats_merged, db_with_wins], ignore_index=True)

    return db_with_wins

In [5]:
to_train = bets_result_scrambler(minimal_bets)

In [6]:
to_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4624 entries, 0 to 4623
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   match_id  4624 non-null   int64         
 1   date      4624 non-null   datetime64[ns]
 2   y         4624 non-null   int64         
 3   name_P1   4624 non-null   object        
 4   bets_P1   4624 non-null   float64       
 5   name_P2   4624 non-null   object        
 6   bets_P2   4624 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 253.0+ KB


In [7]:
df = to_train.drop(columns=['match_id', 'date', 'name_P1', 'name_P2'])



In [8]:
from sklearn.model_selection import train_test_split

Y = pd.DataFrame(df['y'])
df = df.drop(['y'], axis=1)
X = df
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=45, stratify=Y)

In [9]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

model = DecisionTreeClassifier()

result = model.fit(X_train, y_train)
y_pred = model.predict(X_test)

basic_report = classification_report(y_test, y_pred, output_dict=True)

basic_report

{'0': {'precision': 0.5817490494296578,
  'recall': 0.6623376623376623,
  'f1-score': 0.6194331983805669,
  'support': 231.0},
 '1': {'precision': 0.61,
  'recall': 0.5258620689655172,
  'f1-score': 0.5648148148148149,
  'support': 232.0},
 'accuracy': 0.593952483801296,
 'macro avg': {'precision': 0.5958745247148289,
  'recall': 0.5940998656515898,
  'f1-score': 0.5921240065976909,
  'support': 463.0},
 'weighted avg': {'precision': 0.595905033300758,
  'recall': 0.593952483801296,
  'f1-score': 0.5920650234620908,
  'support': 463.0}}

AVG preccision for bets is 66%


In [6]:
minimal_bets.to_csv('../db/clean_and_no_NaN/minimal_bets.csv', index=False)

In [7]:
from features import player_name_for_bets


all_matches_w_bets = pd.DataFrame()

for id, match in all_matches[30000:30550].iterrows():
    player1 = player_name_for_bets(match['P1_name'])
    player2 = player_name_for_bets(match['P2_name'])
    print(player1, player2)
    date = match['date']
    match_bets = minimal_bets[(minimal_bets['date'] == date) & (
        minimal_bets['Winner'] == player1)]
    if match_bets.empty:
        continue
    else:

        match['P1_bets'] = match_bets['B365W'].values[0]
        match['P2_bets'] = match_bets['B365L'].values[0]
        all_matches_w_bets = pd.concat([all_matches_w_bets, match], axis=1)

    print(match)

Monaco J. Gicquel M.
Massu N. Sweeting R.
Montanes A. Koubek S.
Troicki V. Devilder N.
Kunitsyn I. Canas G.
Reynolds B. Chela J.
Querrey S. Muller G.
Becker B. Levine J.
Hewitt L. Sela D.
match_id                         30009
tourney_name             Miami Masters
draw_size                           96
date               2009-03-25 00:00:00
P1_name                 Lleyton Hewitt
P1_hand                              R
P1_ht                            180.0
P1_age                            28.0
P2_name                      Dudi Sela
P2_hand                              R
P2_ht                            175.0
P2_age                            23.9
round                             R128
minutes                          118.0
w_ace                              4.0
w_df                               6.0
w_svpt                            77.0
w_1stIn                           39.0
w_1stWon                          25.0
w_2ndWon                          19.0
w_SvGms                         

In [8]:
all_matches.head()

,match_id,tourney_name,draw_size,date,P1_name,P1_hand,P1_ht,P1_age,P2_name,P2_hand,...,P2_rank_points,tourney_level_A,tourney_level_D,tourney_level_F,tourney_level_G,tourney_level_M,surface_Carpet,surface_Clay,surface_Grass,surface_Hard
0,1,Auckland,32,2000-01-10,Tommy Haas,R,188.0,21.7,Jeff Tarango,L,...,595.0,1,0,0,0,0,0,0,0,1
1,2,Auckland,32,2000-01-10,Juan Balcells,R,190.0,24.5,Franco Squillari,L,...,723.0,1,0,0,0,0,0,0,0,1
2,3,Auckland,32,2000-01-10,Alberto Martin,R,175.0,21.3,Alberto Berasategui,R,...,649.0,1,0,0,0,0,0,0,0,1
3,4,Auckland,32,2000-01-10,Juan Carlos Ferrero,R,183.0,19.9,Roger Federer,R,...,616.0,1,0,0,0,0,0,0,0,1
4,5,Auckland,32,2000-01-10,Michael Sell,R,180.0,27.3,Nicolas Escude,R,...,873.0,1,0,0,0,0,0,0,0,1


In [9]:
minimal_bets.head()

,date,Winner,Loser,B365W,B365L
7297,2002-06-10,Johansson T.,Pretzsch A.,1.100,6.500
7298,2002-06-10,Kafelnikov Y.,Bjorkman J.,1.444,2.625
7300,2002-06-10,Koubek S.,Moya C.,2.625,1.444
7303,2002-06-10,Pavel A.,Stepanek R.,1.533,2.375
7306,2002-06-10,Sampras P.,Stoliarov A.,1.143,5.000


In [10]:
# all_bets.describe(include='all')

# all_bets.to_csv(f'../db/clean_and_no_NaN/atp_bets.csv', index=False)

In [11]:
for id, bets in minimal_bets[15450:15500].iterrows():
    name = bets['Winner'].split(' ')
    last_name = name[-1]
    search_pattern = f'r{last_name}$'
    match = all_matches[all_matches['date'] == bets['date']]

In [24]:
test_bets = minimal_bets[minimal_bets['date'] == '2008-06-17']

In [25]:
print(test_bets)

            date         Winner          Loser  B365W  B365L
24652 2008-06-17     Gicquel M.     Gasquet R.   6.00   1.11
24653 2008-06-17      Melzer J.     Andreev I.   1.80   1.90
24654 2008-06-17       Ancic M.      Darcis S.   1.11   6.00
24655 2008-06-17      Ferrer D.  Dell'Acqua M.   1.06   8.00
24674 2008-06-17       Groth S.       Baker J.   1.50   2.50
24676 2008-06-17       Simon G.       Young D.   1.61   2.20
24677 2008-06-17  Bogdanovic A.        Ward J.   1.38   2.87
24678 2008-06-17    Stepanek R.       Isner J.   1.33   3.25
24680 2008-06-17     Monfils G.    Guccione C.   1.57   2.25
24681 2008-06-17      Spadea V.   Slabinsky A.   1.16   4.50
24682 2008-06-17    Verdasco F.      Daniel M.   1.11   6.00


In [27]:
test_matches = all_matches[all_matches['date'] == '2008-06-17']
print(test_matches)

Empty DataFrame
Columns: [match_id, tourney_name, draw_size, date, P1_name, P1_hand, P1_ht, P1_age, P2_name, P2_hand, P2_ht, P2_age, round, minutes, w_ace, w_df, w_svpt, w_1stIn, w_1stWon, w_2ndWon, w_SvGms, w_bpSaved, w_bpFaced, l_ace, l_df, l_svpt, l_1stIn, l_1stWon, l_2ndWon, l_SvGms, l_bpSaved, l_bpFaced, P1_rank, P1_rank_points, P2_rank, P2_rank_points, tourney_level_A, tourney_level_D, tourney_level_F, tourney_level_G, tourney_level_M, surface_Carpet, surface_Clay, surface_Grass, surface_Hard]
Index: []

[0 rows x 45 columns]


In [17]:
all_matches.head()

,match_id,tourney_name,draw_size,date,P1_name,P1_hand,P1_ht,P1_age,P2_name,P2_hand,...,P2_rank_points,tourney_level_A,tourney_level_D,tourney_level_F,tourney_level_G,tourney_level_M,surface_Carpet,surface_Clay,surface_Grass,surface_Hard
0,1,Auckland,32,2000-01-10,Tommy Haas,R,188.0,21.7,Jeff Tarango,L,...,595.0,1,0,0,0,0,0,0,0,1
1,2,Auckland,32,2000-01-10,Juan Balcells,R,190.0,24.5,Franco Squillari,L,...,723.0,1,0,0,0,0,0,0,0,1
2,3,Auckland,32,2000-01-10,Alberto Martin,R,175.0,21.3,Alberto Berasategui,R,...,649.0,1,0,0,0,0,0,0,0,1
3,4,Auckland,32,2000-01-10,Juan Carlos Ferrero,R,183.0,19.9,Roger Federer,R,...,616.0,1,0,0,0,0,0,0,0,1
4,5,Auckland,32,2000-01-10,Michael Sell,R,180.0,27.3,Nicolas Escude,R,...,873.0,1,0,0,0,0,0,0,0,1


In [15]:
matches_ex = all_matches[(all_matches['P1_name'].str.contains(
    'Soderling$') == True) & (all_matches['date'] == '2003-07-07')]

In [16]:
print(matches_ex)

       match_id tourney_name  draw_size       date          P1_name P1_hand  \
10187     10188       Bastad         32 2003-07-07  Robin Soderling       R   

       P1_ht  P1_age       P2_name P2_hand  ...  P2_rank_points  \
10187  193.0    18.8  Markus Hipfl       R  ...            91.0   

       tourney_level_A tourney_level_D  tourney_level_F  tourney_level_G  \
10187                1               0                0                0   

       tourney_level_M  surface_Carpet  surface_Clay  surface_Grass  \
10187                0               0             1              0   

       surface_Hard  
10187             0  

[1 rows x 45 columns]


In [1]:
from ratings import elo_rating

ModuleNotFoundError: No module named 'ratings'